In [ ]:
!pip install --no-index --no-deps /kaggle/input/aes-whls/aes_whls/huggingface_hub-0.23.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/aes-whls/aes_whls/safetensors-0.4.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install --no-index --no-deps /kaggle/input/aes-whls/aes_whls/sentence_transformers-2.8.0.dev0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/aes-whls/aes_whls/tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install --no-index --no-deps /kaggle/input/aes-whls/aes_whls/transformers-4.40.2-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/aes-whls/aes_whls/textstat-0.7.3-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/aes-whls/aes_whls/pyphen-0.15.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/aes-whls/aes_whls/einops-0.8.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/aes-whls/aes_whls/pyspellchecker-0.8.1-py3-none-any.whl

In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tokenizers import (
    decoders,
    models,
    pre_tokenizers,
    normalizers,
    processors,
    trainers,
    Tokenizer
)

from datasets import Dataset
from tqdm.auto import tqdm
from transformers import PreTrainedTokenizerFast

import gc

import spacy
from collections import Counter

import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer
import textstat
from spellchecker import SpellChecker

from sentence_transformers import SentenceTransformer, models
from sklearn.linear_model import Ridge
from sklearn.multioutput import MultiOutputRegressor

import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
import torch

import copy
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments,DataCollatorWithPadding
from datasets import Dataset
from glob import glob
import re
import random
import warnings
warnings.filterwarnings('ignore')
import gc
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import re
import random
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier,GradientBoostingClassifier,BaggingClassifier
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB,MultinomialNB,ComplementNB
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.metrics import cohen_kappa_score
from lightgbm import log_evaluation, early_stopping
from sklearn.linear_model import SGDClassifier
import pickle
from scipy.special import softmax
from sklearn.preprocessing import OneHotEncoder

import transformers as T
import torch
from tqdm import tqdm
from sklearn.cluster import KMeans
device = "cuda" if torch.cuda.is_available() else "cpu"

nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('vader_lexicon')

tqdm.pandas()

nlp = spacy.load("en_core_web_sm")

# **Load data**

In [ ]:
competition_dir = '/kaggle/input/learning-agency-lab-automated-essay-scoring-2'
persaude_dir = 'kaggle/input/persaude-corpus-2'
train = pd.read_csv(competition_dir+'/train.csv')
test = pd.read_csv(competition_dir+'/test.csv')
persuade = pd.read_csv('/kaggle/input/persaude-corpus-2/persuade_2.0_human_scores_demo_id_github.csv')
sample_submission = pd.read_csv(competition_dir+'/sample_submission.csv')

# **textstat**

In [ ]:
def textstat_features(text):
    features = {}
    features['flesch_reading_ease'] = textstat.flesch_reading_ease(text)
    features['flesch_kincaid_grade'] = textstat.flesch_kincaid_grade(text)
    features['smog_index'] = textstat.smog_index(text)
    features['coleman_liau_index'] = textstat.coleman_liau_index(text)
    features['automated_readability_index'] = textstat.automated_readability_index(text)
    features['dale_chall_readability_score'] = textstat.dale_chall_readability_score(text)
    features['difficult_words'] = textstat.difficult_words(text)
    features['linsear_write_formula'] = textstat.linsear_write_formula(text)
    features['gunning_fog'] = textstat.gunning_fog(text)
    features['text_standard'] = textstat.text_standard(text, float_output=True)
    features['spache_readability'] = textstat.spache_readability(text)
    features['mcalpine_eflaw'] = textstat.mcalpine_eflaw(text)
    features['reading_time'] = textstat.reading_time(text)
    features['syllable_count'] = textstat.syllable_count(text)
    features['lexicon_count'] = textstat.lexicon_count(text)
    features['monosyllabcount'] = textstat.monosyllabcount(text)

    return features

train['textstat_features'] = train['full_text'].apply(textstat_features)
train_textstat = pd.DataFrame(train['textstat_features'].tolist())

test['textstat_features'] = test['full_text'].apply(textstat_features)
test_textstat = pd.DataFrame(test['textstat_features'].tolist())

train_textstat.head()

# **linguistic**

In [ ]:
def extract_linguistic_features(text):

    doc = nlp(text)
    features = {}

    # NER Features
    entity_counts = {"GPE": 0, "PERCENT": 0, "NORP": 0, "ORG": 0, "CARDINAL": 0, "MONEY": 0, "DATE": 0,
                    "LOC": 0, "PERSON": 0, "QUANTITY": 0, "EVENT": 0, "ORDINAL": 0, "WORK_OF_ART": 0,
                    "LAW": 0, "PRODUCT": 0, "TIME": 0, "FAC": 0, "LANGUAGE": 0}
    for entity in doc.ents:
        if entity.label_ in entity_counts:
            entity_counts[entity.label_] += 1
    features['NER_Features'] = entity_counts

    # POS Features
    pos_counts = {"ADJ": 0, "NOUN": 0, "VERB": 0, "SCONJ": 0, "PRON": 0, "PUNCT": 0, "DET": 0, "AUX": 0,
                "PART": 0, "ADP": 0, "SPACE": 0, "CCONJ": 0, "PROPN": 0, "NUM": 0, "ADV": 0,
                "SYM": 0, "INTJ": 0, "X": 0}
    for token in doc:
        if token.pos_ in pos_counts:
            pos_counts[token.pos_] += 1
    features['POS_Features'] = pos_counts

    # tag Features
    tags = {"RB": 0, "-RRB-": 0, "PRP$": 0, "JJ": 0, "TO": 0, "VBP": 0, "JJS": 0, "DT": 0, "''": 0, "UH": 0, "RBS": 0, "WRB": 0, ".": 0,
        "HYPH": 0, "XX": 0, "``": 0, "SYM": 0, "VB": 0, "VBN": 0, "WP": 0, "CC": 0, "LS": 0, "POS": 0, "NN": 0, ",": 0, "NNPS": 0,
          "RP": 0, ":": 0, "$": 0, "PDT": 0, "VBZ": 0, "VBD": 0, "JJR": 0, "-LRB-": 0, "IN": 0, "RBR": 0, "WDT": 0, "EX": 0, "MD": 0,
            "_SP": 0, "NNP": 0, "CD": 0, "VBG": 0, "NNS": 0, "PRP": 0}

    for token in doc:
        if token.tag_ in tags:
            tags[token.tag_] += 1
    features['tag_Features'] = tags

    # tense features
    tenses = [i.morph.get("Tense") for i in doc]
    tenses = [i[0] for i in tenses if i]
    tense_counts = Counter(tenses)
    features['past_tense_ratio'] = tense_counts.get("Past", 0) / (tense_counts.get("Pres", 0) + tense_counts.get("Past", 0) + 1e-5)
    features['present_tense_ratio'] = tense_counts.get("Pres", 0) / (tense_counts.get("Pres", 0) + tense_counts.get("Past", 0) + 1e-5)


    # len features

    features['word_count'] = len(doc)
    features['sentence_count'] = len([sentence for sentence in doc.sents])
    features['words_per_sentence'] = features['word_count'] / features['sentence_count']
    features['std_words_per_sentence'] = np.std([len(sentence) for sentence in doc.sents])

    features['unique_words'] = len(set([token.text for token in doc]))
    features['lexical_diversity'] = features['unique_words'] / features['word_count']

    paragraph = text.split('\n\n')

    features['paragraph_count'] = len(paragraph)

    features['avg_chars_by_paragraph'] = np.mean([len(paragraph) for paragraph in paragraph])
    features['avg_words_by_paragraph'] = np.mean([len(nltk.word_tokenize(paragraph)) for paragraph in paragraph])
    features['avg_sentences_by_paragraph'] = np.mean([len(nltk.sent_tokenize(paragraph)) for paragraph in paragraph])

    # sentiment features
    analyzer = SentimentIntensityAnalyzer()
    sentences = nltk.sent_tokenize(text)

    compound_scores, negative_scores, positive_scores, neutral_scores = [], [], [], []
    for sentence in sentences:
        scores = analyzer.polarity_scores(sentence)
        compound_scores.append(scores['compound'])
        negative_scores.append(scores['neg'])
        positive_scores.append(scores['pos'])
        neutral_scores.append(scores['neu'])

    features["mean_compound"] = np.mean(compound_scores)
    features["mean_negative"] = np.mean(negative_scores)
    features["mean_positive"] = np.mean(positive_scores)
    features["mean_neutral"] = np.mean(neutral_scores)

    features["std_compound"] = np.std(compound_scores)
    features["std_negative"] = np.std(negative_scores)
    features["std_positive"] = np.std(positive_scores)
    features["std_neutral"] = np.std(neutral_scores)

    return features

train['linguistic_features'] = train['full_text'].progress_apply(extract_linguistic_features)

train_linguistic = pd.json_normalize(train['linguistic_features'])



test['linguistic_features'] = test['full_text'].progress_apply(extract_linguistic_features)

test_linguistic = pd.json_normalize(test['linguistic_features'])

train_linguistic.head()

In [ ]:
tag_cols = [col for col in train_linguistic.columns if col.startswith('tag')]
col_cols = [col for col in train_linguistic.columns if col.startswith('col')]
pos_cols = [col for col in train_linguistic.columns if col.startswith('pos')]

for col in tag_cols:
    train_linguistic[f"{col}_ratio"] = train_linguistic[col] / train_linguistic['word_count']
    test_linguistic[f"{col}_ratio"] = test_linguistic[col] / test_linguistic['word_count']

for col in col_cols:
    test_linguistic[f"{col}_ratio"] = test_linguistic[col] / test_linguistic['word_count']

for col in pos_cols:
    test_linguistic[f"{col}_ratio"] = test_linguistic[col] / test_linguistic['word_count']

train_linguistic.head()

In [ ]:
merged_df = pd.concat([train_textstat, train_linguistic], axis=1)

merged_df_test = pd.concat([test_textstat, test_linguistic], axis=1)

# **SpellCheck**

In [ ]:
spell = SpellChecker()

def spell_check(text):
    words = nltk.word_tokenize(text)
    misspelled = spell.unknown(words)

    mispelled_count = len(misspelled)
    misspelled_ratio = mispelled_count / len(words)

    return mispelled_count, misspelled_ratio

train['spell_check_features'] = train['full_text'].progress_apply(spell_check)

spell_check_df = pd.DataFrame(train['spell_check_features'].tolist(), columns=['misspelled_count', 'misspelled_ratio'])

test['spell_check_features'] = test['full_text'].progress_apply(spell_check)

test_check_df = pd.DataFrame(test['spell_check_features'].tolist(), columns=['misspelled_count', 'misspelled_ratio'])

spell_check_df.head()

In [ ]:
merged_df = pd.concat((merged_df, spell_check_df), axis=1)

merged_df_test = pd.concat((merged_df_test, test_check_df), axis=1)

# **feedback**

In [ ]:
feedback_path = '/kaggle/input/feedback-data'
sentpath = '/kaggle/input/sent-debsmall'
feedback_df = pd.read_csv(feedback_path+'/feedback_data.csv')

feed_embeds = []

merged_embeds = []

test_embeds = []

for i in range(5):
    model_path = sentpath + f'/deberta_small_trained/temp_fold{i}_checkpoints'
    word_embedding_model = models.Transformer(model_path, max_seq_length=1024)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    model.half()
    model = model.to(device)

    feed_custom_embeddings_train = model.encode(feedback_df.loc[:, 'full_text'].values, device=device,
                                                show_progress_bar=True, normalize_embeddings=True)

    feed_embeds.append(feed_custom_embeddings_train)

    merged_custom_embeddings = model.encode(train.loc[:, 'full_text'].values, device=device,
                                            show_progress_bar=True, normalize_embeddings=True)

    merged_embeds.append(merged_custom_embeddings)


    test_custom_embeddings = model.encode(test.loc[:, 'full_text'].values, device=device,
                                            show_progress_bar=True, normalize_embeddings=True)

    test_embeds.append(test_custom_embeddings)

feed_embeds = np.mean(feed_embeds, axis=0)
merged_embeds = np.mean(merged_embeds, axis=0)
test_embeds = np.mean(test_embeds, axis=0)




In [ ]:
targets = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']


ridge = Ridge(alpha=1.0)

multioutputregressor = MultiOutputRegressor(ridge)



multioutputregressor.fit(feed_embeds, feedback_df.loc[:, targets])

In [ ]:
feedback_predictions = multioutputregressor.predict(merged_embeds)

feedback_predictions_df = pd.DataFrame(feedback_predictions, columns=targets)

test_feedback_predictions = multioutputregressor.predict(test_embeds)

test_feedback_predictions_df = pd.DataFrame(test_feedback_predictions, columns=targets)

feedback_predictions_df.head()

In [ ]:
merged_df = pd.concat((merged_df, feedback_predictions_df), axis=1)

merged_df_test = pd.concat((merged_df_test, test_feedback_predictions_df), axis=1)

# **topic**

In [ ]:
tokenizer = T.BertTokenizer.from_pretrained("/kaggle/input/bert-base-uncased", cache_dir="./cache/")
model = T.AutoModel.from_pretrained("/kaggle/input/bert-base-uncased").to(device)
# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained(
#     "/kaggle/input/deberta-v3-large/deberta-v3-large",
#     cache_dir="./cache/"
# )
# model = AutoModel.from_pretrained(
#     "/kaggle/input/deberta-v3-large/deberta-v3-large"
# ).to(device)

In [ ]:
def text_to_bert_vector(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)['input_ids'].to(device)
    with torch.no_grad():
        outputs = model(inputs)
    cls_vector = outputs.last_hidden_state[:, 0, :]
    return cls_vector.squeeze().detach().cpu().numpy()

def transfer_bert(full_text_list,batch_size = 100):
    bert_vectors = []
    for i in tqdm(range(0, len(full_text_list), batch_size)):
        batch_text = full_text_list[i:i + batch_size]
        try:
            bert_vector_batch = text_to_bert_vector(batch_text)
            bert_vectors.append(bert_vector_batch)
        except Exception as e:
            batch_text = full_text_list[i:]
            bert_vector_batch = text_to_bert_vector(batch_text)
            bert_vectors.append(bert_vector_batch)
    bert_vectors = [vec for vec in bert_vectors if vec is not None]
    bert_vectors_array = np.vstack(bert_vectors)
    return bert_vectors_array

In [ ]:
bert_path = '/kaggle/input/bert-cls'
persuade_bert = np.load(bert_path + '/persuade_bert.npy')
train_bert = np.load(bert_path + '/train_bert.npy')

In [ ]:
# bert_path = '/kaggle/input/deberta'
# persuade_bert = np.load(bert_path + '/persuade_deberta.npy')
# train_bert = np.load(bert_path + '/train_deberta.npy')

In [ ]:
#persuade_bert = transfer_bert(persuade['full_text'].to_list())
#train_bert = transfer_bert(train['full_text'].to_list())
test_bert = transfer_bert(test['full_text'].to_list(),batch_size=4)

In [ ]:
# #kmeans
# n_clusters = 15
# kmeans = KMeans(n_clusters=n_clusters, random_state=42)
# kmeans.fit(persuade_bert)
# train_topic = kmeans.predict(train_bert)
# test_topic = kmeans.predict(test_bert)

# train_topic_one_hot = np.eye(n_clusters)[train_topic]
# test_topic_one_hot = np.eye(n_clusters)[test_topic]

# topic_col = [f'topic_{i}' for i in range(15)]
# train_topic_df = pd.DataFrame(train_topic_one_hot, columns=topic_col)
# test_topic_df = pd.DataFrame(test_topic_one_hot, columns=topic_col)
# merged_df = pd.concat((merged_df, train_topic_df), axis=1)
# merged_df_test = pd.concat((merged_df_test, test_topic_df), axis=1)

In [ ]:
#knn
knn_clf = KNeighborsClassifier(n_neighbors=15)
knn_clf.fit(persuade_bert, persuade['prompt_name'])

encoder = OneHotEncoder(sparse=False)
encoded_data = encoder.fit(persuade['prompt_name'].values.reshape(-1, 1))

train_topic = knn_clf.predict(train_bert)
test_topic = knn_clf.predict(test_bert)

train_topic_one_hot = encoder.transform(train_topic.reshape(-1, 1))
test_topic_one_hot = encoder.transform(test_topic.reshape(-1, 1))

topic_col = encoder.get_feature_names_out(input_features=['Topic'])

train_topic_df = pd.DataFrame(train_topic_one_hot, columns=topic_col)
test_topic_df = pd.DataFrame(test_topic_one_hot, columns=topic_col)

merged_df = pd.concat((merged_df, train_topic_df), axis=1)
merged_df_test = pd.concat((merged_df_test, test_topic_df), axis=1)

# **Paragraph feature**

In [ ]:
columns = [
    (
        pl.col("full_text").str.split(by="\n\n").alias("paragraph")
    ),
]

# 载入训练集和测试集，同时对full_text数据使用\n\n字符分割为列表，重命名为paragraph
# Load training and testing sets, while using \ n \ n character segmentation to list and renaming to paragraph for full_text data
train_df = pl.read_csv(competition_dir+"/train.csv").with_columns(columns)
test_df = pl.read_csv(competition_dir+"/test.csv").with_columns(columns)

In [ ]:
cList = {
  "ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because",  "could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not",
  "haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will","he'll've": "he will have","he's": "he is",
  "how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is","I'd": "I would","I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have",
  "isn't": "is not","it'd": "it had","it'd've": "it would have","it'll": "it will", "it'll've": "it will have","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not",
  "might've": "might have","mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have","needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not","oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
  "shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
  "should've": "should have","shouldn't": "should not","shouldn't've": "should not have","so've": "so have","so's": "so is","that'd": "that would","that'd've": "that would have","that's": "that is","there'd": "there had","there'd've": "there would have","there's": "there is","they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we had",
  "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
  "weren't": "were not","what'll": "what will","what'll've": "what will have",
  "what're": "what are","what's": "what is","what've": "what have","when's": "when is","when've": "when have",
  "where'd": "where did","where's": "where is","where've": "where have","who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is",
  "why've": "why have","will've": "will have","won't": "will not","won't've": "will not have","would've": "would have","wouldn't": "would not",
  "wouldn't've": "would not have","y'all": "you all","y'alls": "you alls","y'all'd": "you all would",
  "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have","you're": "you are",  "you've": "you have"
   }

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
def dataPreprocessing(x):
    x = x.lower()
    x = removeHTML(x)
    x = re.sub("@\w+", '',x)
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    x = re.sub("http\w+", '',x)
    x = re.sub(r"\s+", " ", x)
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
    x = x.strip()
    return x

In [ ]:
# 段落特征
# paragraph features
def Paragraph_Preprocess(tmp):
    # 将段落列表扩展为一行行的数据
    # Expand the paragraph list into several lines of data
    tmp = tmp.explode('paragraph')
    # 段落预处理
    # Paragraph preprocessing
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(dataPreprocessing))
    # 计算每一个段落的长度
    # Calculate the length of each paragraph
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x)).alias("paragraph_len"))
    # 计算每一个段落中句子的数量和单词的数量
    # Calculate the number of sentences and words in each paragraph
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x.split('.'))).alias("paragraph_sentence_cnt"),
                    pl.col('paragraph').map_elements(lambda x: len(x.split(' '))).alias("paragraph_word_cnt"),)
    return tmp
# feature_eng
paragraph_fea = ['paragraph_len','paragraph_sentence_cnt','paragraph_word_cnt']
def Paragraph_Eng(train_tmp):
    aggs = [
        # 统计段落长度大于和小于 i 值的个数
        # Count the number of paragraph lengths greater than and less than the i-value
        *[pl.col('paragraph').filter(pl.col('paragraph_len') >= i).count().alias(f"paragraph_{i}_cnt") for i in [50,75,100,125,150,175,200,250,300,350,400,500,600,700] ],
        *[pl.col('paragraph').filter(pl.col('paragraph_len') <= i).count().alias(f"paragraph_{i}_cnt") for i in [25,49]],
        # 其他
        # other
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in paragraph_fea],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in paragraph_fea],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in paragraph_fea],
        *[pl.col(fea).first().alias(f"{fea}_first") for fea in paragraph_fea],
        *[pl.col(fea).last().alias(f"{fea}_last") for fea in paragraph_fea],
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df
tmp = Paragraph_Preprocess(train_df)
train_feats = Paragraph_Eng(tmp)
train_feats['score'] = train_df['score']
# 获取特征名称
# Obtain feature names
feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))

tmp = Paragraph_Preprocess(test_df)
test_feats = Paragraph_Eng(tmp)

# **Sentence feature**

In [ ]:
# sentence feature
def Sentence_Preprocess(tmp):
    # 对full_text预处理，并且使用句号分割出文本的句子
    # Preprocess full_text and use periods to segment sentences in the text
    tmp = tmp.with_columns(pl.col('full_text').map_elements(dataPreprocessing).str.split(by=".").alias("sentence"))
    tmp = tmp.explode('sentence')
    # 计算句子的长度
    # Calculate the length of a sentence
    tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x)).alias("sentence_len"))
    # 筛选出句子长度大于15的那一部分数据
    # Filter out the portion of data with a sentence length greater than 15
    tmp = tmp.filter(pl.col('sentence_len')>=15)
    # 统计每一句中单词的数量
    # Count the number of words in each sentence
    tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x.split(' '))).alias("sentence_word_cnt"))

    return tmp
# feature_eng
sentence_fea = ['sentence_len','sentence_word_cnt']
def Sentence_Eng(train_tmp):
    aggs = [
        # 统计句子长度大于 i 的句子个数
        # Count the number of sentences with a length greater than i
        *[pl.col('sentence').filter(pl.col('sentence_len') >= i).count().alias(f"sentence_{i}_cnt") for i in [15,50,100,150,200,250,300] ],
        # 其他
        # other
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in sentence_fea],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in sentence_fea],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in sentence_fea],
        *[pl.col(fea).first().alias(f"{fea}_first") for fea in sentence_fea],
        *[pl.col(fea).last().alias(f"{fea}_last") for fea in sentence_fea],
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

tmp = Sentence_Preprocess(train_df)
# 将新生成的特征数据和之前生成的特征数据合并
# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(Sentence_Eng(tmp), on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))

tmp = Sentence_Preprocess(test_df)
test_feats = test_feats.merge(Sentence_Eng(tmp), on='essay_id', how='left')

# **Word feature**

In [ ]:
# word feature
def Word_Preprocess(tmp):
    # 对full_text预处理，并且使用空格符分割出文本的单词
    # Preprocess full_text and use spaces to separate words from the text
    tmp = tmp.with_columns(pl.col('full_text').map_elements(dataPreprocessing).str.split(by=" ").alias("word"))
    tmp = tmp.explode('word')
    # 计算每一个的单词长度
    # Calculate the length of each word
    tmp = tmp.with_columns(pl.col('word').map_elements(lambda x: len(x)).alias("word_len"))
    # 删除单词长度为0的数据
    # Delete data with a word length of 0
    tmp = tmp.filter(pl.col('word_len')!=0)

    return tmp
# feature_eng
def Word_Eng(train_tmp):
    aggs = [
        # 统计单词长度大于 i+1 的单词个数
        # Count the number of words with a length greater than i+1
        *[pl.col('word').filter(pl.col('word_len') >= i+1).count().alias(f"word_{i+1}_cnt") for i in range(15) ],
        # 其他
        # other
        pl.col('word_len').max().alias(f"word_len_max"),
        pl.col('word_len').mean().alias(f"word_len_mean"),
        pl.col('word_len').std().alias(f"word_len_std"),
        pl.col('word_len').quantile(0.25).alias(f"word_len_q1"),
        pl.col('word_len').quantile(0.50).alias(f"word_len_q2"),
        pl.col('word_len').quantile(0.75).alias(f"word_len_q3"),
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

tmp = Word_Preprocess(train_df)
# 将新生成的特征数据和之前生成的特征数据合并
# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(Word_Eng(tmp), on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))

tmp = Word_Preprocess(test_df)
test_feats = test_feats.merge(Word_Eng(tmp), on='essay_id', how='left')

In [ ]:
import joblib
modelpath = '/kaggle/input/aes2-400-20240419134941'
deberta_oof = joblib.load(modelpath+'/oof.pkl')
print(deberta_oof.shape, train_feats.shape)

for i in range(6):
    train_feats[f'deberta_oof_{i}'] = deberta_oof[:, i]

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))

train_feats.shape

# **Pretrain model score**

In [ ]:
MAX_LENGTH = 1024
MODEL_PATH = modelpath + '/*/*'
EVAL_BATCH_SIZE = 1
models = glob(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(models[0])

def tokenize(sample):
    return tokenizer(sample['full_text'], max_length=MAX_LENGTH, truncation=True)

ds = Dataset.from_pandas(test).map(tokenize).remove_columns(['essay_id', 'full_text'])

args = TrainingArguments(
    ".",
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    report_to="none"
)

predictions = []
for model in models:
    model = AutoModelForSequenceClassification.from_pretrained(model)
    trainer = Trainer(
        model=model,
        args=args,
        data_collator=DataCollatorWithPadding(tokenizer),
        tokenizer=tokenizer
    )
    preds = trainer.predict(ds).predictions
    predictions.append(softmax(preds, axis=-1))
    del model, trainer
    torch.cuda.empty_cache()
    gc.collect()

predicted_score = 0.
for p in predictions:
    predicted_score += p

predicted_score /= len(predictions)

# Meta Features(deberta-v3-large)
for i in range(6):
    test_feats[f'deberta_oof_{i}'] = predicted_score[:, i]

# Features number
feature_names = list(filter(lambda x: x not in ['essay_id','score'], test_feats.columns))
print('Features number: ',len(feature_names))

In [ ]:
merged_df = pd.concat((merged_df, train_feats), axis=1)

merged_df_test = pd.concat((merged_df_test, test_feats), axis=1)

# **LGBM**

In [ ]:
merged_df = merged_df.drop(['essay_id','score'], axis=1)
merged_df_test = merged_df_test.drop(['essay_id'], axis=1)

In [ ]:
def quadratic_weighted_kappa(y_true, y_pred):
    y_true = y_true + a
    y_pred = (y_pred + a).clip(1, 6).round()
    qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic")
    return 'QWK', qwk, True


# metric and objective based on public notebooks

def qwk_obj(y_true, y_pred):
    labels = y_true + a
    preds = y_pred + a
    preds = preds.clip(1, 6)
    f = 1/2*np.sum((preds-labels)**2)
    g = 1/2*np.sum((preds-a)**2+b)
    df = preds - labels
    dg = preds - a
    grad = (df/g - f*dg/g**2)*len(labels)
    hess = np.ones(len(labels))
    return grad, hess
a = 2.998
b = 1.092



skf = StratifiedKFold(n_splits=15, shuffle=True, random_state=42)

scores = []

train['oof'] = 0

test_preds = []

for fold, (train_idx, valid_idx) in enumerate(skf.split(train['full_text'], train['score'])):
    print(f"Fold: {fold}")
    print(f"Train size: {len(train_idx)}")
    print(f"Valid size: {len(valid_idx)}")
    print()


    X_train = merged_df.iloc[train_idx].values
    X_valid = merged_df.iloc[valid_idx].values


    y_train = train['score'].astype('float32').values[train_idx]
    y_valid = train['score'].astype('float32').values[valid_idx]


    y_train = y_train -a
    y_valid = y_valid -a



    model = lgb.LGBMRegressor(
                objective = qwk_obj,
                metrics = 'None',
                learning_rate = 0.01,
                n_estimators=10000,
                random_state=42,
                extra_trees=True,
                class_weight='balanced',
                verbosity = - 1)

    callbacks = [lgb.early_stopping(500, verbose=True, first_metric_only=True), lgb.log_evaluation(period=500)]


    predictor = model.fit(X_train,
                                  y_train,
                                  eval_names=['train', 'valid'],
                                  eval_set=[(X_train, y_train), (X_valid, y_valid)],
                                  eval_metric=quadratic_weighted_kappa,
                                  callbacks=callbacks)

    valid_preds = predictor.predict(X_valid)

    train.loc[valid_idx, 'oof'] = valid_preds + a

    score = quadratic_weighted_kappa(y_valid, valid_preds)
    scores.append(score[1])

    test_preds.append(predictor.predict(merged_df_test) + a)

    print(f"Train QWK: {score}")

print(f"Mean QWK: {np.mean(scores)}")

# **thresholds**

In [ ]:
optimal_thresholds = [1.5, 2.5, 3.5, 4.5, 5.5]

def classify_with_thresholds(oof_clip, optimal_thresholds):

    # Класифікуємо значення oof за допомогою оптимальних порогів
    classified_oof = np.empty_like(oof_clip)
    for i, threshold in enumerate(optimal_thresholds):
        if i == 0:
            classified_oof[oof_clip < threshold] = 1

        else:
            classified_oof[(oof_clip >= optimal_thresholds[i-1]) & (oof_clip < threshold)] = i + 1

        if i == 4:
            classified_oof[oof_clip >= threshold] = 6

    return classified_oof.astype(int)

In [ ]:
optimal_thresholds = [1.5, 2.5, 3.5, 4.5, 5.5]
oof_clip = train['oof'].clip(1, 6)
classified_oof =classify_with_thresholds(oof_clip, optimal_thresholds)
best_kappa_score = cohen_kappa_score(train['score'].astype(int).values, classified_oof, weights='quadratic')

for i in range(5):
    i_threshold=optimal_thresholds[i]-0.5
    for j in range(1000):
        thresholds=[x for x in optimal_thresholds]
        thresholds[i]=i_threshold+j/1000

        classified_oof = classify_with_thresholds(oof_clip, thresholds)
        threshold_kappa_score=cohen_kappa_score(train['score'].astype(int).values, classified_oof, weights='quadratic')

        if threshold_kappa_score>=best_kappa_score:
            best_kappa_score=threshold_kappa_score
            optimal_thresholds[i]=thresholds[i]

optimal_thresholds

In [ ]:
final_preds = (np.mean(test_preds, axis=0))
light_gbm_preds = np.clip(final_preds, 1, 6)
light_gbm_preds = classify_with_thresholds(light_gbm_preds, optimal_thresholds)

# **output**

In [ ]:
sample_submission['score'] = light_gbm_preds
sample_submission.to_csv('submission.csv', index=False)